In [3]:
from config import (
    azure_endpoint, azure_subscription_key, azure_application, 
    azure_compCode, azure_userID, azure_userNM, azure_serviceType, 
    sender_email
)

import json
import requests 

In [13]:
def evaluate_article(keyword, article_summary):
    """
    키워드와 기사 요약을 평가하여 점수(float)와 해설(str)을 분리하여 반환하는 함수
    
    Args:
        keyword (str): 검색 키워드
        article_summary (str): 기사 요약 텍스트
        
    Returns:
        tuple: (점수(float), 해설(str))
    """
    headers = {
        "Ocp-Apim-Subscription-Key": azure_subscription_key,
        "application": azure_application,
        "compCode": azure_compCode,
        "userID": azure_userID,
        "userNM": azure_userNM,
        "serviceType": azure_serviceType,
        "Content-Type": "application/json; charset=utf-8"
    }
    
    user_content = f"""당신의 역할은 다음과 같습니다:
- "키워드"와 "기사 요약"을 입력값으로 받습니다.
- 기사 요약이 키워드와 얼마나 직접적이고 깊이 있게 연관되는지(키워드 연관성), 그리고 기사 내용이 포장재 회사에 실질적으로 도움이 되는 정보인지(실용성)를 각각 10점 만점으로 평가합니다.
- 두 점수의 평균을 내어 통합점수(0.5점 단위, 소수 첫째자리까지)를 산출합니다.
- 통합점수 기준으로 해당 기사의 '유효성'(유효/참고용/유효하지 않음)을 판단합니다.
- 최종 리턴은 반드시 "X.X점" 형식으로 시작하고, 그 다음 줄부터 해설을 작성하세요.

평가 기준은 다음과 같습니다:
- 키워드 연관성: 기사 요약이 키워드를 직접적으로, 깊이 있게 다루면 10점, 거의 무관하면 1점.
- 실용성: 기사 내용이 포장재 회사 실무에 직접적으로 도움이 되면 10점, 전혀 도움되지 않으면 1점.
- 통합점수 = (키워드 연관성 + 실용성) ÷ 2 (소수 첫째자리)
- 유효성: 통합점수 8점 이상=유효, 5~7.5점=참고용, 5점 미만=유효하지 않음.

---
키워드: {keyword}
기사 요약: {article_summary}
---"""
    
    data = {
        "messages": [
            {"role": "system", "content": "당신은 포장재 산업 전문 분석가입니다. 기사의 연관성과 실용성을 객관적으로 평가합니다."},
            {"role": "user", "content": user_content}
        ],
        "temperature": 0.3
    }
    
    try:
        data_str = json.dumps(data, ensure_ascii=False).encode("utf-8")
        response = requests.post(azure_endpoint, headers=headers, data=data_str)
        response.raise_for_status()
        result = response.json()
        evaluation = result["choices"][0]["message"]["content"].strip()
        
        # 점수와 해설 분리 - 여러 패턴 시도
        import re
        
        # 패턴 1: "X.X점" 형식으로 시작하는 경우
        score_match = re.match(r'^(\d+(\.\d+)?)[점점]\s*(.*)', evaluation, re.DOTALL)
        if score_match:
            score = float(score_match.group(1))
            explanation = score_match.group(3).strip()
            return score, explanation
            
        # 패턴 2: 숫자로 시작하고 줄바꿈이나 대시(—)로 구분된 경우
        score_match = re.match(r'^(\d+(\.\d+)?)\s*[\n\r—\-–]*\s*(.*)', evaluation, re.DOTALL)
        if score_match:
            score = float(score_match.group(1))
            explanation = score_match.group(3).strip()
            return score, explanation
        
        # 그 외의 경우: 첫 줄에서 숫자만 추출
        first_line = evaluation.split('\n', 1)[0].strip()
        score_match = re.search(r'(\d+(\.\d+)?)', first_line)
        if score_match:
            score = float(score_match.group(1))
            # 첫 줄을 제외한 나머지를 설명으로
            if '\n' in evaluation:
                explanation = evaluation.split('\n', 1)[1].strip()
            else:
                # 숫자 부분을 제외한 나머지를 설명으로
                explanation = first_line.replace(score_match.group(0), '').strip()
                if explanation.startswith('점'):
                    explanation = explanation[1:].strip()
            return score, explanation
            
        # 모든 시도 실패 시
        print(f"파싱 실패. 원본 응답: {evaluation}")
        return 0.0, evaluation
        
    except Exception as e:
        print("평가 중 오류 발생:", e)
        return 0.0, f"평가 오류 발생: {str(e)}"

In [15]:
# 테스트 코드
keyword = "Korean rice wine Market"
article_summary = """
Wine Market Forecast - Expected to Grow from $314.34 Billion to $422.08 Billion by 2032, at 4.30% CAGR| E. & J. Gallo Winery, Concha y Toro, The Wine Group, Bacardi Limited
와인 시장 전망 - 2032년까지 3,143억 4천만 달러에서 4,220억 8천만 달러로 성장할 것으로 예상되며, 연평균 성장률(CAGR)은 4.30%입니다. | E. & J. Gallo Winery, Concha y Toro, The Wine Group, Bacardi Limited
업로드 시간: 2025-04-21

원문 URL: openpr.com

요약:

글로벌 와인 시장, 2025년 3,143억 4천만 달러에서 2032년 4,220억 8천만 달러로 성장 예상
연평균 성장률(CAGR) 4.30%
주요 기업: E. & J. Gallo Winery, Concha y Toro, The Wine Group, Bacardi Limited 등
시장 세분화: 제품 유형(스틸 와인, 스파클링 와인, 강화 와인) 및 색상 유형(레드 와인, 화이트 와인, 로제 와인)
지역 분석: 북미, 유럽, 아시아 태평양, 라틴 아메리카, 중동 및 아프리카
보고서의 주요 이점: 시장 동향, 기회, 경쟁 분석, 지역별 수익 매핑 등 제공
"""

score, response = evaluate_article(keyword, article_summary)
print(score)
print(response)

3.0
기사 요약은 글로벌 와인 시장에 대한 전반적인 성장 전망을 다루고 있지만, "Korean rice wine Market"이라는 특정 키워드와는 직접적인 연관성이 부족합니다. 한국의 쌀 와인 시장에 대한 정보는 포함되어 있지 않으며, 전반적인 와인 시장에 대한 내용이 주를 이루고 있습니다. 따라서 키워드 연관성은 2점으로 평가합니다.  
실용성 측면에서도, 포장재 회사에 대한 직접적인 정보나 실무에 도움이 되는 내용이 부족하여 4점으로 평가합니다.  
통합점수는 (2 + 4) ÷ 2 = 3.0점으로, 유효성은 '유효하지 않음'으로 판단됩니다.


In [8]:
!pip install exchangelib

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 22.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/620.8 kB ? eta -:--:--
   ---------------------------------------- 620.8/620.8 kB 5.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import smtplib
import urllib.parse
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.header import Header
from email.encoders import encode_base64

from exchangelib import Credentials, Account, Message, HTMLBody, Configuration, DELEGATE

def send_email_exchange(subject, html_body, recipient, attachment_path=None, attachment_filename=None):
    """
    Exchange 메일 서버를 사용하여 이메일을 발송하는 함수
    
    Args:
        subject: 이메일 제목
        html_body: HTML 형식의 이메일 본문
        recipient: 수신자 이메일 주소
        attachment_path: 첨부 파일 경로 (선택사항)
        attachment_filename: 첨부 파일로 표시될 이름 (선택사항)
    
    Returns:
        bool: 발송 성공 여부
    """
    sender = os.getenv("EXCHANGE_EMAIL")
    email_pw = os.getenv("EXCHANGE_PASSWORD")
    exchange_server = os.getenv("EXCHANGE_SERVER")
    
    if None in [sender, email_pw, exchange_server]:
        print("Exchange 서버 환경변수가 설정되어 있지 않습니다.")
        return False
    
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = recipient
    msg["Subject"] = subject
    msg.attach(MIMEText(html_body, "html"))
    
    # 첨부 파일 처리
    if attachment_path is not None:
        try:
            # 첨부 파일 읽기
            with open(attachment_path, "rb") as attachment_file:
                part = MIMEBase("application", "pdf")
                part.set_payload(attachment_file.read())
            
            # 파일 인코딩
            encode_base64(part)
            
            # 첨부 파일 이름 설정
            if attachment_filename is None:
                attachment_filename = os.path.basename(attachment_path)
            
            # 파일명에 한글 또는 특수문자가 포함된 경우에 대비
            try:
                # UTF-8로 인코딩된 파일명 생성
                encoded_filename = attachment_filename.encode('utf-8')
                
                # Content-Disposition 헤더 설정 (RFC2231 표준)
                part.add_header(
                    'Content-Disposition',
                    'attachment',
                    filename='utf-8\'\'%s' % urllib.parse.quote(encoded_filename)
                )
                
                # 이전 버전 호환을 위한 추가 헤더
                part.add_header(
                    'Content-Disposition',
                    'attachment',
                    filename=attachment_filename
                )
            except Exception as encode_error:
                print(f"파일명 인코딩 중 오류: {encode_error}. 기본 방식으로 대체합니다.")
                part.add_header(
                    "Content-Disposition", 
                    f'attachment; filename="{attachment_filename}"'
                )
            
            msg.attach(part)
            print(f"첨부 파일 추가: {attachment_filename}")
            
        except Exception as e:
            print("첨부 파일 추가 중 오류 발생:", e)
            return False
    
    try:
        # Exchange 서버 연결 설정
        smtp_server = exchange_server
        smtp_port = 587  # Exchange 기본 포트 (환경에 따라 다를 수 있음)
        
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.ehlo()
        server.starttls()
        server.ehlo()
        server.login(sender, email_pw)
        server.sendmail(sender, recipient, msg.as_string())
        server.quit()
        print("Exchange 이메일 발송 성공!")
        return True
    except Exception as e:
        print("Exchange 이메일 발송 중 오류 발생:", e)
        return False

def send_email_ews():
    # 기본 인증 사용 (NTLM은 별도로 지정)
    credentials = Credentials(
        username=os.getenv("EXCHANGE_EMAIL"),
        password=os.getenv("EXCHANGE_PASSWORD")
    )
    
    # 서버 정보 및 인증 방식 설정
    config = Configuration(
        server=os.getenv("EXCHANGE_SERVER"),
        credentials=credentials
    )
    
    # 계정 연결
    account = Account(
        primary_smtp_address=os.getenv("EXCHANGE_EMAIL"),
        config=config,
        autodiscover=False,  # 직접 서버 지정 시 False
        access_type=DELEGATE
    )
    
    # 메시지 작성
    message = Message(
        account=account,
        subject='테스트 메일',
        body=HTMLBody('<html><body>EWS로 보낸 테스트 메일입니다.</body></html>'),
        to_recipients=['cbj6214@dongwon.com']
    )
    
    # 전송
    message.send()
    return True
# 테스트 함수
def test_exchange_email():
    """
    Exchange 이메일 발송 기능 테스트
    """
    # 환경 변수 설정 (실제 사용 시에는 .env 파일이나 시스템 환경 변수 사용 권장)
    os.environ["EXCHANGE_EMAIL"] = "cbj6214@dongwon.com"
    os.environ["EXCHANGE_PASSWORD"] = ""
    os.environ["EXCHANGE_SERVER"] = "email.dongwon.com"
    
    # 테스트 이메일 내용
    subject = "Exchange 이메일 테스트"
    html_body = """
    <html>
    <body>
        <h1>Exchange 이메일 테스트</h1>
        <p>이 이메일은 Exchange 서버를 통해 발송되었습니다.</p>
    </body>
    </html>
    """
    recipient = "cbj6214@dongwon.com"
    
    # 첨부 파일 테스트 (선택사항)
    # attachment_path = "test_document.pdf"
    
    # 이메일 발송 시도
    #result2 = send_email_exchange(subject, html_body, recipient) # , attachment_path)
    result = send_email_ews()
    return result

if __name__ == "__main__":
    test_result = test_exchange_email()
    print(f"테스트 결과: {'성공' if test_result else '실패'}")

테스트 결과: 성공
